In [ ]:
import numpy as np
import xrayutilities as xu

#### using x-ray utils
## https://xrayutilities.sourceforge.io/examples.html

In [ ]:
#### methods for defining crystal structures for the 2nd part of the program
def Imma():
    # create orthorhombic unit cell with space-group 62,
    # here for simplicity without base
    l = xu.materials.SGLattice(74, 7.68880, 7.68880, 7.68880)
    return l

def Pmbar3m(a):
    # create cubic unit cell with space-group 221,
    # here for simplicity without base
    l = xu.materials.SGLattice(221, a)
    return l

def Pbar3m1():
    # create cubic unit cell with space-group 164, trigonal only a and c are parameters.
    # here for simplicity without base
    l = xu.materials.SGLattice(164,  5.69135,7.43999 )
    return l

In [ ]:
#### Set the HKL of the reflection you want to check is accessible#####
H = 0
K = 0
L = 2
# set the x-ray energy in keV
energy = 9.52e3 


In [ ]:
### XRAY Utilities #####

### set up system with the material
system = xu.materials.Crystal("system", Imma()) ### input your crystal type as second argument
### setup up q convention system.
### standard 4 circle goniometer
# omega, chi, phi, theta
qconv=xu.QConversion(('x+', 'y+','z-'), ('x+'),(0,1,0))

### setup experiment, this sets scattering plane
### Non Coplanar geometry
hxrd = xu.NonCOP(system.Q(1,0, 0), system.Q(0, 0, 1), en=energy,qconv=qconv, sampleor='z+')


In [ ]:
#### as an introduction and a sanity check, use x-ray utils to calculate a set of goniometer angles for the wavevector q in the coplanar geometry.
#### Verify these angles by fitting goniometer angles to your reflection 
hkl=(H, K, L)
q_material = system.Q(hkl)

### put the q of your reflection in the crystal into the experiment 


q_exp = hxrd.Transform(q_material) 

### calculate the goniometer angles from q
print("Goniometer Angles from q in the Coplanar geometry")
om, chi, phi, tt = hxrd.Q2Ang(q_exp, trans=False)
print(np.round([om,chi,phi,tt],2))
bounds = ((-180,180), (-180,180),(-180,180) , (-180,180)) 
#bounds = (om,chi,phi,tt)
#ang, qerror, errcode = xu.Q2AngFit(q_exp, hxrd, bounds) ### fit the angles to the HKL
ang, qerror, errcode = xu.Q2AngFit(q_exp, hxrd, bounds) 
print("Fit angles: omega, chi, phi, twotheta:") 
print(qerror)
print(ang)
print("Fit: sanity check with back-transformation (hkl): ",
      np.round(hxrd.Ang2HKL(*ang,mat=system),2))

ang_calc = [om, chi, phi, tt]
print("Calculated Angles: sanity check with back-transformation (hkl): ",
      np.round(hxrd.Ang2HKL(*ang_calc,mat=system),2))

print(q_exp)

In [ ]:
### For grazing exit geometry, the position of the detector two-theta is about a degree larger than the incidence angle omega. 
### This constraint is contained in the Euler angle fit bounds, which solves for Euler angles in the non-coplanar which correspond to the HKL reflection 
### input at the beginning of the program
### prints the Euler angles when the fit error is low
for i in range(0, 90, 1):
    bounds = (i, (-180,180),(-180,180) , i+1) 
    ang, qerror, errcode = xu.Q2AngFit(q_exp, hxrd, bounds) ### fit the angles to the HKL
    if(qerror<0.1):
        print(ang, qerror)
        print(np.round(hxrd.Ang2HKL(*ang,mat=system),2))

In [ ]:
### Part of the program that checks whether a reflection is accessible for thin film geometry
### For a reflection to be accessible, in the dot produce of the incident wave vector and the film surface normal must be negative
### For a reflection to be accessible, in the dot produce of the outgoing wave vector and the film surface normal must be positive
import math
### angles found for the fit using grazing exit constraints
ang = np.array([   om, chi, phi, tt] )

# set parameters for the calculation of k_in and k_out in the LAB frame.
z = np.array([0,0,1])  # z normal to the sample in experiment frame, no angle change
y = np.array([0,1,0])  #  in-plane for the experiment frame, no angle change
x = np.array([1,0,0])  #  x scattering plane in the experiment frame


q = q_exp
k = hxrd.k0
tth = np.deg2rad(ang[3])
chi = np.deg2rad(ang[1])
om = np.deg2rad(ang[0])
phi = np.deg2rad(ang[2])





#### calculate the k_in and k_out vectors in the experiment frame
beta = tth - om
print(np.rad2deg(om))
print(np.rad2deg(tth))
print(np.rad2deg(beta))
ki = k * (np.array([np.cos(om)])[:, np.newaxis] * y[np.newaxis, :] - np.array([np.sin(om)])[:, np.newaxis] * z[np.newaxis, :])
kd = k * (np.array([np.cos(beta)])[:, np.newaxis] * y[np.newaxis, :] + np.array([np.sin(beta)])[:, np.newaxis] * z[np.newaxis, :])


#### change incident and outgoing wavector the film surface normal (FSN) to the LAB frame, which has been moved by the angles in ang, kin and kout have not been changed


ki_sf = qconv.transformSample2Lab(ki,ang[0], 0, 0, 0)
kd_sf = qconv.transformSample2Lab(kd, ang[0], 0, 0, 0)

FSN = qconv.transformSample2Lab(z, *ang)


In [ ]:
#### normalize the vectors (make them unit vectors)
kd_norm = kd_sf[0]/np.linalg.norm(kd_sf[0])
ki_norm = ki_sf[0]/np.linalg.norm(ki_sf[0])

In [ ]:
print("FSN (film surface normal) dot k_in and k_out. IF they have the same sign, not accessible")
print("FSN dot k_i")
print(np.dot(FSN,kd_norm))
print("FSN dot k_out")
print(np.dot(FSN,ki_norm))